In [1]:
## Following MATLAB code from http://capture-clarity.org/clarity-based-tractography/

In [2]:
import os

In [3]:
## Parameters (the script loops through all parameters and saves each result automatically)
dogsigmaArr = [1];  # Sigma values for derivative of gaussian filter, recommended value: 0.6 - 1.3 (based on actual data)
gausigmaArr = [2.3];  # Sigma values for gaussian filter, recommended value: 1.3 - 2.3 (based on actual data)
angleArr = [25];  # Angle thresholds for fiber tracking, recommended value: 20 - 30

In [4]:
pwd

u'/Users/Tony/Documents/Git Folder/seelviz/Tony/ipynb'

In [5]:
import numpy as np

In [6]:
import math
from scipy import ndimage
import nibabel as nib

In [7]:
## Change later on
file_path = "/Users/Tony/Documents/Git Folder/seelviz/Tony/ipynb/TIFF_stack"
directory = os.path.dirname(file_path)

In [8]:
cd TIFF_stack/

/Users/Tony/Documents/Git Folder/seelviz/Tony/ipynb/TIFF_stack


In [9]:
from PIL import Image
from numpy import matlib
from scipy import signal

In [10]:
'''
Function to generate derivatives of Gaussian kernels, in either 1D, 2D, or 3D.
Source code in MATLAB obtained from Qiyuan Tian, Stanford University, September 2015
Edited to work in Python by Tony.
'''

def doggen(sigma):
    halfsize = np.ceil(3 * np.max(sigma))
    x = range(np.single(-halfsize), np.single(halfsize + 1));  # Python colon is not inclusive at end, while MATLAB is.
    dim = len(sigma);
    
    if dim == 1:
        X = np.array(x);  # Remember that, by default, numpy arrays are elementwise multiplicative
        k = -X * np.exp(-X**2/(2 * sigma**2));
        
    elif dim == 2:
        [X, Y] = np.meshgrid(x, x);
        k = -X * np.exp(-X**2/(2*sigma[0]^2) * np.exp(-Y**2))
        
    elif dim == 3:
        [X, Y, Z] = np.meshgrid(x, x, x);
        X = X.transpose(0, 2, 1);  # Obtained through vigorous testing (see below...)
        Y = Y.transpose(2, 0, 1);
        Z = Z.transpose(2, 1, 0);
        
        X = X.astype(float);
        Y = Y.astype(float);
        Z = Z.astype(float);
        k = -X * np.exp(np.divide(-np.power(X, 2), 2 * np.power(sigma[0], 2))) * np.exp(np.divide(-np.power(Y,2), 2 * np.power(sigma[1],2))) * np.exp(np.divide(-np.power(Z,2), 2 * np.power(sigma[2],2)))
        
    else:
        print 'Only supports up to 3 dimensions'
        
    return np.divide(k, np.sum(np.abs(k[:])));

In [11]:
'''
Function to generate Gaussian kernels, in 1D, 2D and 3D.
Source code in MATLAB obtained from Qiyuan Tian, Stanford University, September 2015
Edited to work in Python by Tony. 
'''

def gaussgen(sigma):
    halfsize = np.ceil(3 * max(sigma));
    x = range(np.single(-halfsize), np.single(halfsize + 1));

    dim = len(sigma);

    if dim == 1:
        k = np.exp(-x**2 / (2 * sigma^2));
    
    elif dim == 2:
        [X, Y] = np.meshgrid(x, x);
        k = np.exp(-X**2 / (2 * sigma[0]**2)) * np.exp(-Y**2 / (2 * sigma[1]**2)); 
    
    elif dim == 3:
        [X, Y, Z] = np.meshgrid(x, x, x);
        X = X.transpose(0, 2, 1);  # Obtained through vigorous testing (see below...)
        Y = Y.transpose(2, 0, 1);
        Z = Z.transpose(2, 1, 0);
        
        X = X.astype(float);  # WHY PYTHON?
        Y = Y.astype(float);
        Z = Z.astype(float);
        k = np.exp(-X**2 / (2 * sigma[0]**2)) * np.exp(-Y**2 / (2 * sigma[1]**2)) * np.exp(-Z**2 / (2 * sigma[2]**2));
    
    else:
        print 'Only supports up to dimension 3'

    return np.divide(k, np.sum(np.abs(k)));

In [12]:
# Set up results directory
if not os.path.exists(directory):
    os.makedirs(directory)

# im = Image.open('page1.tiff')  # Needs to be changed to dynamically go down list of fnDataArr (currently just loads same test image)
# Omitted: channel data (red/green - our CLARITY data was single channel, so no channel data loaded.)
ones = np.single(100 * np.ones((10, 10, 10)));  #data is hard coded to be np.ones

np.random.seed(1)

norm = np.single(20 + 2 * np.random.randn(10, 10, 10)) # image histogram is normally distributed with mean 20, std 2

uniform = np.single(100 * np.random.rand(10, 10, 10)) # image histogram is normally distributed between 0 and 100

# image has a line of intensity value 100 at col = 4, depth = 4
line = np.zeros((10, 10, 10))
line[:,2,2] = 100
line = np.single(line)

img_data = norm

In [13]:
types_of_data = [ones, norm, line];
tensorFSLTEST = [];

for x in types_of_data:
    img_data = x;
    for jj in range(len(dogsigmaArr)):
        dogsigma = dogsigmaArr[jj];
        print "Start DoG Sigma on " + str(dogsigma);

        # Generate dog kernels
        dogkercc = doggen([dogsigma, dogsigma, dogsigma]);
        dogkercc = np.transpose(dogkercc, (0, 2, 1));  # annoying

        #print dogkercc.shape;
        #print dogkercc[:, :, 0];

        dogkerrr = np.transpose(dogkercc, (1, 0, 2));

        #print dogkerrr[:, :, 0];
        dogkerzz = np.transpose(dogkercc, (0, 2, 1));

        #print dogkerzz[:, :, 0];

        # Compute gradients
        grr = signal.fftconvolve(img_data, dogkercc, 'same');
        grr = np.transpose(grr, (1, 0, 2));

        #print grr[:, :, 0];

        gcc = signal.fftconvolve(img_data, dogkerrr, 'same');
        gcc = np.transpose(gcc, (1, 0, 2));

        #print gcc[:, :, 0];

        gzz = signal.fftconvolve(img_data, dogkerzz, 'same');
        gzz = np.transpose(gzz, (1, 0, 2));

        #print gzz[:, :, 0];

        # Compute gradient products
        gprrrr = np.multiply(grr, grr);

        #print gprrrr[:, :, 0];

        gprrcc = np.multiply(grr, gcc);

        #print gprrcc[:, :, 0];

        gprrzz = np.multiply(grr, gzz);

        #print gprrzz[:, :, 0]

        gpcccc = np.multiply(gcc, gcc);
        gpcczz = np.multiply(gcc, gzz);
        gpzzzz = np.multiply(gzz, gzz);

        # Compute gradient amplitudes
        # print ga.dtype;
        ga = np.sqrt(gprrrr + gpcccc + gpzzzz);

        #print ga[:, :, 0];

        print "GA SHAPE:"
        print ga.shape;

        # Convert numpy ndarray object to Nifti data type
        gradient_amplitudes_data = nib.Nifti1Image(ga, affine=np.eye(4));

        # Save gradient amplitudes image 
        nib.save(gradient_amplitudes_data, 'gradient_amplitudes.nii');

        # Compute gradient vectors
        gv = np.concatenate((grr[..., np.newaxis], gcc[..., np.newaxis], gzz[..., np.newaxis]), axis = 3);
        gv = np.divide(gv, np.tile(ga[..., None], [1, 1, 1, 3]));
        #print gv[:, :, 0, 1];

        print "GV SHAPE:"
        print gv.shape;

        # Convert numpy ndarray object to Nifti data type
        gradient_vectors_data = nib.Nifti1Image(gv, affine=np.eye(4));

        # Save gradient vectors
        nib.save(gradient_vectors_data, 'gradient_vectors.nii');

        # Compute structure tensor
        for kk in range(len(gausigmaArr)):
            gausigma = gausigmaArr[kk];
            print "Start Gauss Sigma with gausigma = " + str(gausigma);

            print "Generating Gaussian kernel..."
            gaussker = np.single(gaussgen([gausigma, gausigma, gausigma]));

            #print gaussker[:, :, 0];

            print "Blurring gradient products..."
            gprrrrgauss = signal.fftconvolve(gprrrr, gaussker, "same");
            gprrccgauss = signal.fftconvolve(gprrcc, gaussker, "same");
            gprrzzgauss = signal.fftconvolve(gprrzz, gaussker, "same");
            gpccccgauss = signal.fftconvolve(gpcccc, gaussker, "same");
            gpcczzgauss = signal.fftconvolve(gpcczz, gaussker, "same");
            gpzzzzgauss = signal.fftconvolve(gpzzzz, gaussker, "same");

            print "Saving a copy for this Gaussian sigma..."
            tensorfsl = np.concatenate((gprrrrgauss[..., np.newaxis], gprrccgauss[..., np.newaxis], gprrzzgauss[..., np.newaxis], gpccccgauss[..., np.newaxis], gpcczzgauss[..., np.newaxis], gpzzzzgauss[..., np.newaxis]), axis = 3);
            tensorFSLTEST.append(tensorfsl);
            
             # Convert numpy ndarray object to Nifti data type
            tensor_fsl_data = nib.Nifti1Image(tensorfsl, affine=np.eye(4));

            nib.save(tensor_fsl_data, "dogsigma_" + str(jj) + "gausigma_" + str(kk) + 'tensorfsl.nii');

    print 'Complete!'

Start DoG Sigma on 1
GA SHAPE:
(10, 10, 10)
GV SHAPE:
(10, 10, 10, 3)
Start Gauss Sigma with gausigma = 2.3
Generating Gaussian kernel...
Blurring gradient products...
Saving a copy for this Gaussian sigma...
Complete!
Start DoG Sigma on 1
GA SHAPE:
(10, 10, 10)
GV SHAPE:
(10, 10, 10, 3)
Start Gauss Sigma with gausigma = 2.3
Generating Gaussian kernel...
Blurring gradient products...
Saving a copy for this Gaussian sigma...
Complete!
Start DoG Sigma on 1
GA SHAPE:
(10, 10, 10)
GV SHAPE:
(10, 10, 10, 3)
Start Gauss Sigma with gausigma = 2.3
Generating Gaussian kernel...
Blurring gradient products...
Saving a copy for this Gaussian sigma...
Complete!


In [14]:
## ONES:
tensorfsl = tensorFSLTEST[0];

In [15]:
## Compare tensor_fsl_data with imported values from MATLAB
print tensorfsl.shape  # numpy

(10, 10, 10, 6)


In [16]:
MATLAB_output = nib.load("/Users/Tony/Documents/Git Folder/seelviz/Tony/ipynb/test_MATLAB_10by10by10_outputs/dogsig1_gausig2.3/test_MATLAB_tensorfsl_dogsig1_gausig2.3.nii")

In [17]:
print MATLAB_output.shape

(10, 10, 10, 6)


In [18]:
MATLAB_np_array = MATLAB_output.get_data()
print MATLAB_np_array.shape

(10, 10, 10, 6)


In [19]:
print tensorfsl[:, :, :, 0];

[[[ 109.85943033  147.37287499  179.26563687  201.37589584  212.3919397
    212.3919397   201.37589584  179.26563687  147.37287499  109.85943033]
  [ 147.37287499  197.69594847  240.47905765  270.13925545  284.91692228
    284.91692228  270.13925545  240.47905765  197.69594847  147.37287499]
  [ 179.26563687  240.47905765  292.52080083  328.5997214   346.57540288
    346.57540288  328.5997214   292.52080083  240.47905765  179.26563687]
  [ 201.37589584  270.13925545  328.5997214   369.12854264  389.32130808
    389.32130808  369.12854264  328.5997214   270.13925545  201.37589584]
  [ 212.3919397   284.91692228  346.57540288  389.32130808  410.61869621
    410.61869621  389.32130808  346.57540288  284.91692228  212.3919397 ]
  [ 212.3919397   284.91692228  346.57540288  389.32130808  410.61869621
    410.61869621  389.32130808  346.57540288  284.91692228  212.3919397 ]
  [ 201.37589584  270.13925545  328.5997214   369.12854264  389.32130808
    389.32130808  369.12854264  328.5997214   

In [20]:
print MATLAB_np_array[:, :, :, 0];

[[[ 109.85939789  147.37295532  179.26557922  201.37585449  212.39183044
    212.39196777  201.37599182  179.26565552  147.37287903  109.85943604]
  [ 147.37289429  197.69589233  240.47901917  270.13919067  284.91690063
    284.91680908  270.13919067  240.47918701  197.69593811  147.37290955]
  [ 179.26560974  240.47901917  292.52096558  328.59942627  346.57531738
    346.57531738  328.59970093  292.52087402  240.47904968  179.26560974]
  [ 201.37588501  270.13912964  328.59945679  369.12838745  389.32110596
    389.32110596  369.12866211  328.59973145  270.13916016  201.37582397]
  [ 212.39186096  284.91671753  346.57528687  389.3210144   410.61871338
    410.61868286  389.32144165  346.57556152  284.91680908  212.391922  ]
  [ 212.39189148  284.91677856  346.57519531  389.32107544  410.6187439
    410.61865234  389.32144165  346.57556152  284.9168396   212.39190674]
  [ 201.37590027  270.13916016  328.59954834  369.12850952  389.3213501
    389.32125854  369.12860107  328.59979248  2

In [21]:
truth_boolean = np.isclose(tensorfsl, MATLAB_np_array, rtol = 1e-4)
print truth_boolean;

[[[[ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   ..., 
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]]

  [[ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   ..., 
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]]

  [[ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   ..., 
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]]

  ..., 
  [[ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   ..., 
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ Tr

In [22]:
print np.sum(truth_boolean == True);  # Total possible = 6000

6000


In [23]:
## NORM:
tensorfsl = tensorFSLTEST[1];
MATLAB_output = nib.load("/Users/Tony/Documents/Git Folder/seelviz/Tony/ipynb/test_MATLAB_norm/dogsig1_norm_gausig2.3/test_MATLAB_norm_tensorfsl_dogsig1_gausig2.3.nii")

In [24]:
MATLAB_np_array = MATLAB_output.get_data()
print MATLAB_np_array.shape

(10, 10, 10, 6)


In [25]:
print tensorfsl[:, :, :, 0];

[[[  4.56324146   6.12466482   7.44329461   8.33924851   8.75785789
     8.71146336   8.21516077   7.27954919   5.96456499   4.4373526 ]
  [  6.1701928    8.26949481  10.03666643  11.23327738  11.78946619
    11.72289234  11.05216321   9.78961606   8.01599581   5.95816383]
  [  7.55722182  10.11335813  12.25817028  13.7058822   14.37624334
    14.29214664  13.47375751  11.93285836   9.76704226   7.25483142]
  [  8.5233428   11.39225957  13.79406644  15.41285942  16.1630889
    16.07093077  15.15521806  13.42463036  10.98723657   8.15818775]
  [  8.98548237  12.00251868  14.527714    16.23268134  17.02947221
    16.94347107  15.98904389  14.17028531  11.59967724   8.61193497]
  [  8.93826754  11.94185738  14.46171602  16.17262591  16.98500321
    16.91845444  15.98061253  14.17144542  11.60345726   8.61432414]
  [  8.40155999  11.2353136   13.62338528  15.25841271  16.05029625
    16.00936057  15.13618466  13.42872332  10.99592941   8.16169978]
  [  7.40805908   9.91975365  12.04774367 

In [26]:
print MATLAB_np_array[:, :, :, 0];

[[[  4.64503241   6.27805614   7.69753361   8.71435738   9.25117588
     9.28773594   8.80899811   7.81494665   6.38361835   4.72023249]
  [  6.1748662    8.34263229  10.22420311  11.56876755  12.27563477
    12.32106495  11.68707848  10.37267971   8.47811317   6.2729187 ]
  [  7.43925714  10.04508305  12.30221176  13.90958595  14.74921799
    14.79718208  14.03501511  12.46073723  10.19070244   7.54487085]
  [  8.28669643  11.18048763  13.68008518  15.4517746   16.36836052
    16.40913582  15.55823135  13.81382751  11.30131531   8.37119865]
  [  8.69358635  11.71907711  14.32400036  16.15963745  17.09663773
    17.11969757  16.21851158  14.39394951  11.77509689   8.7235136 ]
  [  8.68278027  11.69599056  14.28234577  16.09355545  17.00299644
    17.00113106  16.08519745  14.2618084   11.6601963    8.63624859]
  [  8.2505722   11.11008453  13.55901432  15.26438618  16.10599327
    16.07888985  15.18825912  13.44803047  10.98403454   8.13071728]
  [  7.37310505   9.92998219  12.11748886

In [27]:
truth_boolean = np.isclose(tensorfsl, MATLAB_np_array, rtol=1)
print truth_boolean;

[[[[ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   ..., 
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]]

  [[ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   ..., 
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]]

  [[ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   ..., 
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]]

  ..., 
  [[ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   ..., 
   [ True  True  True  True  True  True]
   [ True  True  True  True  True  True]
   [ Tr

In [28]:
print np.sum(truth_boolean == True);  # Total possible = 6000

5859


In [29]:
## LINE:
tensorfsl = tensorFSLTEST[2];
MATLAB_output = nib.load("/Users/Tony/Documents/Git Folder/seelviz/Tony/ipynb/test_MATLAB_line/dogsig1_norm_gausig2.3/test_MATLAB_norm_tensorfsl_dogsig1_gausig2.3.nii")

In [30]:
MATLAB_np_array = MATLAB_output.get_data()
print MATLAB_np_array.shape

(10, 10, 10, 6)


In [31]:
print tensorfsl[:, :, :, 0];

[[[  4.83643911e+00   6.26751423e+00   6.83320450e+00   6.26774552e+00
     4.83684502e+00   3.14043409e+00   1.71557751e+00   7.88569287e-01
     3.04817989e-01   9.57210025e-02]
  [  6.48792675e+00   8.40766775e+00   9.16652296e+00   8.40797803e+00
     6.48847127e+00   4.21279084e+00   2.30139178e+00   1.05784021e+00
     4.08903480e-01   1.28406631e-01]
  [  7.89197009e+00   1.02271596e+01   1.11502376e+01   1.02275370e+01
     7.89263245e+00   5.12447514e+00   2.79943282e+00   1.28676596e+00
     4.97393722e-01   1.56194934e-01]
  [  8.86534962e+00   1.14885567e+01   1.25254853e+01   1.14889806e+01
     8.86609367e+00   5.75651747e+00   3.14470917e+00   1.44547306e+00
     5.58741251e-01   1.75459700e-01]
  [  9.35031868e+00   1.21170254e+01   1.32106780e+01   1.21174725e+01
     9.35110344e+00   6.07142134e+00   3.31673698e+00   1.52454605e+00
     5.89306568e-01   1.85058027e-01]
  [  9.35031868e+00   1.21170254e+01   1.32106780e+01   1.21174725e+01
     9.35110344e+00   6.07142

In [32]:
print MATLAB_np_array[:, :, :, 0];

[[[  5.26199698e-01   6.81899428e-01   7.43445933e-01   6.81924462e-01
     5.26243806e-01   3.41676176e-01   1.86653152e-01   8.57955441e-02
     3.31639014e-02   1.04143489e-02]
  [  6.81899488e-01   8.83669734e-01   9.63427186e-01   8.83702159e-01
     6.81956470e-01   4.42776203e-01   2.41882831e-01   1.11181967e-01
     4.29769158e-02   1.34958960e-02]
  [  7.43445933e-01   9.63426888e-01   1.05038357e+00   9.63462889e-01
     7.43508041e-01   4.82739955e-01   2.63714552e-01   1.21216975e-01
     4.68559042e-02   1.47140063e-02]
  [  6.81924582e-01   8.83702099e-01   9.63462710e-01   8.83734524e-01
     6.81981742e-01   4.42792594e-01   2.41891727e-01   1.11186072e-01
     4.29784879e-02   1.34963933e-02]
  [  5.26243806e-01   6.81956649e-01   7.43508160e-01   6.81981623e-01
     5.26287913e-01   3.41704875e-01   1.86668798e-01   8.58027712e-02
     3.31666842e-02   1.04152225e-02]
  [  3.41676295e-01   4.42776114e-01   4.82740015e-01   4.42792535e-01
     3.41704756e-01   2.21859

In [57]:
truth_boolean = np.isclose(tensorfsl, MATLAB_np_array, rtol=1e-4)
print truth_boolean;

[[[[False  True False False False  True]
   [False  True False False False  True]
   [False  True False False False  True]
   ..., 
   [False  True False False False  True]
   [False  True False False False  True]
   [False  True False False False  True]]

  [[False False False False False False]
   [False False False False False False]
   [False False False False False False]
   ..., 
   [False False False False False False]
   [False False False False False False]
   [False False False False False False]]

  [[False False False False False False]
   [False False False False False False]
   [False False False False False False]
   ..., 
   [False False False False False False]
   [False False False False False False]
   [False False False False False False]]

  ..., 
  [[False False False False False False]
   [False False False False False False]
   [False False False False False False]
   ..., 
   [False False False False False False]
   [False False False False False False]
   [Fal

In [34]:
print np.sum(truth_boolean == True);  # Total possible = 6000

3040


In [63]:
tensorfsl_rotated = np.transpose(tensorfsl, (2, 0, 1, 3));
truth_boolean = np.isclose(tensorfsl_rotated, MATLAB_np_array, rtol=1e-4)
print np.sum(truth_boolean == True);  # Total possible = 6000

200
